In [102]:
import numpy as np
import cupy as cp
from matplotlib import pyplot as plt

use_gpu = False  # Cambiar a False si no tienes GPU o CuPy instalado
array_lib = cp if use_gpu else np

# Consideraciones inciales.

### Funcion para construir la grilla inicial

In [103]:
# Función que construye la grilla inicial

def create_grid(d, grid_size, m, pattern='random'):
    """
    Crea una grilla n-dimensional
    
    Parámetros:
    - d: número de dimensiones
    - grid_size: tamaño de cada dimensión (int o lista de int)
    - m: valor máximo de estado (estados van de 0 a m + 1)
    - pattern: tipo de patrón inicial ('random', 'central', etc.)
    """
    # Convertir grid_size a una tupla si es un solo número
    if isinstance(grid_size, int):
        grid_size = tuple([grid_size] * d)
    else:
        grid_size = tuple(grid_size)
    
    # Crear la grilla según el patrón solicitado
    if pattern == 'random':
        # Grilla con valores aleatorios entre 0 y m
        return array_lib.random.randint(0, m+1, grid_size)
    
    elif pattern == 'central':
        # Grilla con valor alto en el centro y ceros alrededor
        grid = array_lib.zeros(grid_size, dtype=int)
        center = tuple(s // 2 for s in grid_size)
        
        # Colocar valor m en el centro
        indices = tuple(slice(c-1, c+2) for c in center)
        grid[indices] = m
        return grid
    
    elif pattern == 'checkerboard':
        # Patrón de tablero de ajedrez
        indices = array_lib.indices(grid_size)
        sum_indices = array_lib.sum(indices, axis=0)
        return (sum_indices % 2) * m
    
    else:
        # Patrón predeterminado: valores 0
        return array_lib.zeros(grid_size, dtype=int)
    


### Valores inciales

In [104]:
d = 4  # 3 dimensiones de la grilla
grid_size = 3  # Tamaño de los arreglos
m = 3  # Estados posibles por casilla
r = 1 # Radio de vecindad
grid = create_grid(d, grid_size, m, pattern='random') # Grilla inicial

t = 10 # Tiempo


In [ ]:
# https://chatgpt.com/c/67e5c856-0f34-8004-b18b-c3cc71f6a27a

def evolucionar_grilla(grilla, m, r):
    """
    Evoluciona la grilla de un autómata celular en d dimensiones sin usar bucles.
    
    Parámetros:
    - grilla: ndarray de numpy o cupy con la configuración actual.
    - m: Número máximo de estados por celda.
    - r: Radio de vecindad.
    
    Retorna:
    - Nueva grilla evolucionada según las reglas del autómata.
    """  # Determina si estamos usando numpy o cupy
    
    # Expandir la grilla en un toroide (bordes periódicos)
    grilla_expandida = np.pad(grilla, r, mode='wrap')
    
    # Construir el kernel para calcular la suma de vecinos
    kernel_shape = (2 * r + 1,) * grilla.ndim  # Vecindad cúbica de radio r
    kernel = np.ones(kernel_shape, dtype=grilla.dtype)
    kernel[(slice(r, r+1),) * grilla.ndim] = 0  # Excluir la celda central
    
    # Convolución para obtener la suma de vecinos
    from cupyx.scipy.ndimage import convolve as cp_convolve
    from scipy.ndimage import convolve as np_convolve
    
    convolve = cp_convolve if np is cp else np_convolve
    suma_vecinos = convolve(grilla_expandida, kernel, mode='constant', cval=0)
    
    # Extraer la parte válida de la convolución
    slices = tuple(slice(r, -r) for _ in range(grilla.ndim))
    suma_vecinos = suma_vecinos[slices]
    
    # Aplicar las reglas de evolución
    SM = m * (kernel.sum() - 1)  # Valor máximo de suma de vecinos
    intervalos = np.linspace(0, SM, 4)  # Dividimos en 3 intervalos
    
    nueva_grilla = np.copy(grilla)
    nueva_grilla[(suma_vecinos >= intervalos[0]) & (suma_vecinos < intervalos[1])] -= 1
    nueva_grilla[(suma_vecinos >= intervalos[1]) & (suma_vecinos < intervalos[2])] += 1
    nueva_grilla[(suma_vecinos >= intervalos[2])] -= 1
    
    return np.clip(nueva_grilla, 0, m)  # Asegurar valores en [0, m]


grilla = evolucionar_grilla(grid, m, r)

grilla



array([[[[3, 1, 2],
         [1, 3, 2],
         [1, 2, 3]],

        [[1, 3, 1],
         [2, 3, 3],
         [2, 2, 1]],

        [[3, 1, 3],
         [2, 2, 2],
         [3, 3, 3]]],


       [[[3, 2, 2],
         [3, 3, 1],
         [3, 3, 1]],

        [[2, 1, 2],
         [2, 3, 3],
         [3, 2, 1]],

        [[3, 1, 1],
         [1, 3, 3],
         [3, 3, 3]]],


       [[[3, 1, 3],
         [3, 1, 3],
         [1, 2, 2]],

        [[3, 3, 1],
         [2, 3, 1],
         [3, 2, 1]],

        [[3, 1, 3],
         [2, 3, 3],
         [3, 3, 2]]]], dtype=int32)

### Funciones principales

### Pasos a seguir

1. for exterior -> paso del tiempo, por cada tick hace algo
    
    ¿Que hace? 

    a. Saca las otras celdas que pertenezcan a la vecindad

    b. Suma todos los valores 

    c. Se verifica con el intervalo si se + - o mantiene el valor de la celda
